# Combine EpiCure and DeXtrusion results

Open an EpiCured movie (treated) and overlay DeXtrusions probibility maps


*This notebook is part of EpiCure release, see https://gitlab.pasteur.fr/gletort/epicure for more informations*

In [ ]:
import epicure.epicuring as epicure
import os
import numpy as np
import napari
import tifffile as tif

In [ ]:
## path to input data, and names of the movie and EpiCured segmentation
data_dir = os.path.join("..", "data", "selection", "t000-177")
filename = "Ecad"
data_img = filename + ".tif"
data_seg = filename + "_labels.tif"

infile = os.path.join(data_dir, data_img)
segfile = os.path.join(data_dir, "epics", data_seg)

In [ ]:
## Load the EpiCure files and movies
epic = epicure.EpiCure()
napari_viewer = napari.Viewer(show=False)
epic.viewer = napari_viewer
epic.load_movie(infile)
epic.verbose = 0  ## minimal printing of information messages
epic.go_epicure("epics", segfile)

### Do tracking if it not had been done yet

In [ ]:
if epic.tracked == 0:
    epic.tracking.do_tracking()       ## Performs tracking with the default parameters. If you have saved preferences, it will use it.
    epic.save_epicures()              ## save the results in the ouput "epics" folder(s)

### Search for extrusions with `Inspect track` option.

It uses all the default parameters except the ones redefined here. So if other inspect options were selected in the preferences, they will also be runned (eg if `flag track apparition` is selected by default, it will be runned here.

In [ ]:
get_extrusions = False  ## Whether to inspect tracks now to search for extruions. Put to False if it has already been done

if get_extrusions:
    epic.inspecting.get_extrusions.setChecked( True )   ## Be sure to search for extrusions
    epic.inspecting.threshold_disparition.setText( "300" ) ## set the threshold of cell area so that a cell disparition is considered extrusion vs potential error
    epic.inspecting.inspect_tracks( subprogress=False)  ## go, look for extrusions (and other suspects if selected in the saved preferences)
    epic.outputing.export_events_type_format( ["extrusion"], "Fiji ROI" )  ## export extrusion events into Fiji ROIs

### Opens up the DeXtrusion probability maps

Add the probability maps from DeXtrusion into the EpiCure viewer so that events of divisions/extrusions are overlayed with the maps

In [ ]:
## path to the folder containing the DeXtrusion results probability maps
dex_data = os.path.join("..", "data", "selection", "t000-177", "dextrusion-run", "results")

for event, evtcol in zip( ["death", "division", "sop"], ["magenta", "cyan", "yellow"] ):
    evtfile = os.path.join( dex_data, filename + "_cell_" + event + "_proba.tif" )
    if os.path.exists( evtfile ):
        print( "Loading " + evtfile )
        evtraw = tif.TiffFile( evtfile )
        evtmov = evtraw.asarray()
        evtraw.close()
        epic.viewer.add_image( evtmov, name="Dext_"+event, blending="additive", colormap=evtcol, scale=(1, epic.epi_metadata["ScaleXY"],epic.epi_metadata["ScaleXY"]) )

### Detect SOP cells from DeXtrusion probability map with EpiCure intensity group option

Use EpiCure feature to classify cells based on their intensity in the probability map of being a SOP cell estimated by DeXtrusion.

DeXtrusion can outputs probability maps of each events (extrusion, division, SOP). 
For SOP, the probability map will be positive in nearly all the frames within the SOP cell.

EpiCure can automatically classify the cells as SOP based on their intensity along the track with the `Group from track intensity` feature in the interface that calls the `group_positive_cells` function in the API.

In [ ]:
## Set the group name that will be attributted to SOP cells
classify_SOP = False

if classify_SOP:
    ind = epic.editing.group_choice.findText( "SOP" )
    if ind < 0:
        epic.editing.group_choice.addItem( "SOP" )
    else:
        epic.editing.group_choice.setCurrentIndex( ind )

    ## Measure the intensity in the "Dext_sop" layer. Use the sum of all intensity within the cell within the track. 
    epic.editing.group_positive_cells( "Dext_sop", "sum", 0, epic.nframes-1, epic.nframes*200*100 )

In [ ]:
epic.viewer.show()

### Check for False positive events of EpiCure

By measuring the mean probability of event from DeXtrusion probability maps for each event detected by EpiCure, we can flag events that have a low probability and thus might be false positive.

Choose the event to check (division or extrusion) by comparing the results.
You can print all the **indexes** of events in EpiCure that have a low probability to be correct from DeXtrusion results by putting the parameter `show_low_proba_event`. In EpiCure, by going into `Inspect>Events`, set the index of event to the desired value and click on `Go to event`, you can see the corresponding suspicious event and check if it was correct or not. This can be an additionnal way to find segmentation/tracking mistakes that were not detected by the inspection features of EpiCure.

In [ ]:
## Compare extrusion or division events
event = "division"     ## name of event in EpiCure (division or extrusion)
evt_name = "division"  ## name of event in DeXtrusion (to load the proba map, "death" or "division")

show_low_proba_event = True
threshold_low_proba = 150

dz = 1
dxy = 10
extrusions = epic.inspecting.get_events_from_type( event )
extr_intensities = []
for extr in extrusions:
    position, cell = epic.inspecting.get_event_infos( extr )
    minz = max( position[0]-dz, 0 )
    maxz = min( position[0]+dz+1, epic.nframes )
    minx = max( position[1]-dxy, 0 )
    maxx = min( position[1]+dxy+1, epic.imgshape2D[0] )
    miny = max( position[2]-dxy, 0 )
    maxy = min( position[2]+dxy+1, epic.imgshape2D[1] )
    dex_proba = epic.viewer.layers["Dext_"+evt_name].data[minz:maxz, minx:maxx, miny:maxy]
    dex_proba = np.max( dex_proba )
    extr_intensities.append( dex_proba )
    if (show_low_proba_event) and (dex_proba < threshold_low_proba):
        print( epic.inspecting.index_from_id(extr) )

### Cumulative distribution of DeXtrusion probabilities of EpiCure's events

Plot the distribution of probabilities from DeXtrusion results measured at each event detected in EpiCure.
If the two pipelines agree, most events should have a near 255 probability.

In [ ]:
## Cumulative distribution of deXtrusion probabilities of extrusion/division events

extr_intensities = np.array( extr_intensities )
histo, xhisto = np.histogram( extr_intensities, bins = 100, normed=True )
dx = xhisto[1] - xhisto[0]
cdf = np.cumsum( histo ) * dx
import matplotlib.pyplot as plt
plt.plot( xhisto[1:], cdf )